<a href="https://colab.research.google.com/github/bhardwaj2-6/Agnet_Techniques/blob/main/%233_custom_customer_support_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# set environment

In [1]:
!pip install  langgraph langchain langchain_groq langchain_core IPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 13.4 MB/s eta 0:00:00


In [2]:
from typing import TypedDict, Dict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

from IPython.display import display, Image
from langchain_core.runnables.graph import MermaidDrawMethod

In [3]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
llm = ChatGroq(model="gemma2-9b-it",api_key=os.environ.get("GROQ_API_KEY"),verbose=True)


# Define State Structure

In [4]:
class State(TypedDict):
  query: str
  category: str
  sentiment: str
  response: str


# Define Node Functions


In [15]:
def categorize(state: State) -> State:
  """Categorize the customer query into Technical, Billing, or General Support"""
  prompt = ChatPromptTemplate.from_template(
      "categorize the following customer query into one of these categories: "
      "Technical, Billing, Genral. Query:{query}"
  )
  chain = prompt | llm | StrOutputParser()
  result_category = chain.invoke({"query": state["query"]})
  state["category"] = result_category
  return state

def analyze_sentiment(state: State) -> State:
  """Analyze the sentiment of the customer query as Positive, Neutral, or Negative."""
  prompt = ChatPromptTemplate.from_template(
  "Analyze the sentiment of the following customer query. "
        "Respond with either 'Positive', 'Neutral', or 'Negative'. Query: {query}"  )
  chain = prompt | llm | StrOutputParser()
  result_sentiment = chain.invoke({"query": state["query"]})
  state["sentiment"] = result_sentiment
  return state

def handle_technical(state: State) -> State:
  """Handle technical queries"""
  prompt = ChatPromptTemplate.from_template(
      "provide a technical support response to the following customer query: {query}"
  )
  chain = prompt | llm | StrOutputParser()
  result_technical = chain.invoke({"query": state["query"]})
  state["response"]= result_technical
  return state

def handle_billing(state: State) -> State:
    """Provide a billing support response to the query."""
    prompt = ChatPromptTemplate.from_template(
        "Provide a billing support response to the following query: {query}"
    )
    chain = prompt | llm | StrOutputParser()
    result_billing = chain.invoke({"query": state["query"]})
    state["response"] = result_billing
    return state


def handle_general(state: State) -> State:
  """Provide a general support response to the query"""
  prompt = ChatPromptTemplate.from_template(
      "provide a general support response to the following query: {query}"
  )
  chain = prompt | llm | StrOutputParser()
  result_general = chain.invoke({"query": state["query"]})
  state["response"]= result_general
  return state

def escalate(state: State) -> State:
    """Escalate the query to a human agent due to negative sentiment."""
    return {"response": "This query has been escalated to a human agent due to its negative sentiment."}

def route_query(state: State) -> str:
    """Route the query based on its sentiment and category."""
    if state["sentiment"] == "Negative":
        return "escalate"
    elif state["category"] == "Technical":
        return "handle_technical"
    elif state["category"] == "Billing":
        return "handle_billing"
    else:
        return "handle_general"

# Create and Configure the Graph
Here we set up the LangGraph, defining nodes and edges to create our customer support workflow.



In [16]:
# Create the graph
workflow = StateGraph(State)

# Add nodes
workflow.add_node("categorize", categorize)
workflow.add_node("analyze_sentiment", analyze_sentiment)
workflow.add_node("handle_technical", handle_technical)
workflow.add_node("handle_billing", handle_billing)
workflow.add_node("handle_general", handle_general)
workflow.add_node("escalate", escalate)

# Add edges
workflow.add_edge("categorize", "analyze_sentiment")
workflow.add_conditional_edges(
    "analyze_sentiment",
    route_query,
    {
        "handle_technical": "handle_technical",
        "handle_billing": "handle_billing",
        "handle_general": "handle_general",
        "escalate": "escalate"
    }
)
workflow.add_edge("handle_technical", END)
workflow.add_edge("handle_billing", END)
workflow.add_edge("handle_general", END)
workflow.add_edge("escalate", END)

# Set entry point
workflow.set_entry_point("categorize")

# Compile the graph
app = workflow.compile()

# Visualize the Graph
This cell generates and displays a visual representation of our LangGraph workflow.



In [17]:
def run_customer_support(query: str) -> Dict[str, str]:
    """Process a customer query through the LangGraph workflow.

    Args:
        query (str): The customer's query

    Returns:
        Dict[str, str]: A dictionary containing the query's category, sentiment, and response
    """
    results = app.invoke({"query": query})
    return {
        "category": results["category"],
        "sentiment": results["sentiment"],
        "response": results["response"]
    }

In [18]:
# escalate

query = "My internet connection keeps dropping. Can you help?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

# handle_technical

query = "I need help talking to chatGPT"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

# handle_billing

query = "where can i find my receipt?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

# handle_general

query = "What are your business hours?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")

Query: My internet connection keeps dropping. Can you help?
Category: **Technical** 

This query is clearly about a technical issue with the customer's internet connection.  

Sentiment: Negative 

Response: I understand you're experiencing internet connection drops, that can be very frustrating!  

To help me pinpoint the issue, could you tell me a little more about what's happening?

* **How often does your internet drop?** (Every few minutes, hours, etc.)
* **Does it happen on all devices, or just one?**
* **What type of internet connection do you have?** (Cable, DSL, Fiber, etc.)
* **Have you made any recent changes to your network setup?** (New router, new devices, etc.)
* **Are there any error messages you see when the connection drops?**

Once I have a better understanding of the situation, I can offer more specific troubleshooting steps. 

In the meantime, some general things you can try are:

* **Restart your modem and router.** This can often resolve temporary connectivity is